## 라이브러리

In [ ]:
# !pip install keras
# !pip install tensorflow
import keras
from keras.layers import TimeDistributed
import numpy as np
import pandas as pd
# from numpy import array
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.layers import Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from keras import optimizers
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/Shareddrives/인공지능_study')

## 데이터 전처리

In [ ]:
import pandas as pd

df = pd.read_excel('최종_classification_결과.xlsx', header=0, engine='openpyxl')
df.columns = ['이용일', '정상구분', '가맹점명', '금액', '카테고리']

df = pd.DataFrame(df)
df['이용일'] = pd.to_datetime(df['이용일'])

# '할인', 'NaN' 행 제거
df = df[df['정상구분'] != '할인']
df = df[df['정상구분'].notna()]

# '취소' 행 처리
cancelled_rows = df[df['정상구분'] == '취소']

for index, row in cancelled_rows.iterrows():
    target_amount = -row['금액']
    same_merchant = df['가맹점명'] == row['가맹점명']
    same_amount = df['금액'] == target_amount
    is_normal = df['정상구분'] == '정상'

    current_month = row['이용일'].month
    last_month = current_month - 1 if current_month > 1 else 12

    # 이번달에서 동일한 '정상' 결재건 찾기
    matched_rows_current_month = df[same_merchant & same_amount & is_normal & (df['이용일'].dt.month == current_month)]
    if not matched_rows_current_month.empty:
        df.drop(matched_rows_current_month.index, inplace=True)
        df.drop(index, inplace=True)
        continue

    # 지난달에서 동일한 '정상' 결재건 찾기
    matched_rows_last_month = df[same_merchant & same_amount & is_normal & (df['이용일'].dt.month == last_month)]
    if not matched_rows_last_month.empty:
        df.drop(matched_rows_last_month.index, inplace=True)
        df.drop(index, inplace=True)

In [ ]:
pd.set_option('display.max_rows', None)
print(df)

In [ ]:
# 카테고리별 one-hot encoding
df_onehot = pd.get_dummies(df['카테고리'])
df_encoded = pd.concat([df['이용일'], df['금액'], df_onehot], axis=1)
print(df_encoded)

            이용일      금액  교육  생활  쇼핑  여행  음식  의료  이동수단  취미생활
0    2022-02-04    4000   0   0   1   0   0   0     0     0
1    2022-02-05    4700   0   0   0   0   1   0     0     0
2    2022-02-05   17100   0   0   0   0   1   0     0     0
3    2022-02-06    5500   0   0   0   0   0   0     1     0
4    2022-02-06   10000   0   1   0   0   0   0     0     0
5    2022-02-07    4800   0   0   1   0   0   0     0     0
6    2022-02-07    4200   0   0   0   0   1   0     0     0
7    2022-02-07    1500   0   0   0   0   1   0     0     0
8    2022-02-08    3000   0   0   1   0   0   0     0     0
9    2022-02-08    7000   0   0   0   0   1   0     0     0
10   2022-02-09   13900   0   0   0   0   1   0     0     0
11   2022-02-10    6000   0   0   0   1   0   0     0     0
12   2022-02-10    1900   0   0   1   0   0   0     0     0
13   2022-02-11    8500   0   0   0   0   1   0     0     0
14   2022-02-12    9500   0   0   0   0   1   0     0     0
15   2022-02-13   14600   0   0   0   0 

In [ ]:
# 카테고리별 one-hot encoding
#df_onehot = pd.get_dummies(df['카테고리'])
#df_encoded = pd.concat([df['이용일'], df['금액'], df_onehot], axis=1)
#print(df)

# 일별, 카테고리별로 소비금액 합산
df_sum = df_encoded.groupby('이용일').apply(lambda x: x.iloc[:, 2:].multiply(x['금액'], axis=0).sum()).reset_index()

# 원하는 카테고리 순서로 컬럼 재정렬
columns_order = ['이용일', '생활', '이동수단', '쇼핑', '취미생활', '교육', '의료', '음식', '여행']
df_sum = df_sum[columns_order]

df = pd.DataFrame(df_sum)
df['이용일'] = pd.to_datetime(df['이용일'])
df = df.set_index('이용일').resample('D').sum().reset_index()
print(df)

           이용일     생활   이동수단      쇼핑    취미생활     교육      의료      음식      여행
0   2022-02-04      0      0    4000       0      0       0       0       0
1   2022-02-05      0      0       0       0      0       0   21800       0
2   2022-02-06  10000   5500       0       0      0       0       0       0
3   2022-02-07      0      0    4800       0      0       0    5700       0
4   2022-02-08      0      0    3000       0      0       0    7000       0
5   2022-02-09      0      0       0       0      0       0   13900       0
6   2022-02-10      0      0    1900       0      0       0       0    6000
7   2022-02-11      0      0       0       0      0       0    8500       0
8   2022-02-12      0      0       0       0      0       0    9500       0
9   2022-02-13      0      0       0   10000      0       0   14600       0
10  2022-02-14  14700      0       0       0      0       0    9000       0
11  2022-02-15      0      0    7900       0      0       0    7000       0
12  2022-02-

## 모델

In [ ]:
#확정 모델
from sklearn.metrics import r2_score
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau
from keras.models import save_model

# 정규화
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df.iloc[:, 1:])
df_scaled = pd.DataFrame(df_scaled, columns=df.columns[1:])

# 2. LSTM 모델 생성
n_features = df_scaled.shape[1]
seq_length = 92  # 3달 데이터를 기반으로 예측


model = Sequential()

# 입력층
model.add(LSTM(128, activation='tanh', return_sequences=True, input_shape=(seq_length, n_features)))
model.add(Dropout(0.2))

# 중간층
model.add(LSTM(64, activation='tanh', return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(32, activation='tanh'))
model.add(Dropout(0.2))

# 출력층
model.add(Dense(n_features))

optimizer = RMSprop(learning_rate = 0.001)
model.compile(optimizer=optimizer, loss='mse')

lr_scheduler = ReduceLROnPlateau(factor=0.5, patience=5, min_lr=1e-6)
"""
model = Sequential()
model.add(LSTM(64, activation='relu', return_sequences=True, input_shape=(seq_length, n_features)))
model.add(LSTM(32, activation='relu'))
model.add(Dense(n_features))
model.compile(optimizer='adam', loss='mse')
"""

# 3. 학습 데이터 준비
X, y = [], []
for i in range(len(df_scaled) - seq_length):
    X.append(df_scaled.iloc[i:i+seq_length].values)
    y.append(df_scaled.iloc[i+seq_length].values)

X = np.array(X)
y = np.array(y)

# 4. 모델 학습
model.fit(X, y, epochs=100, batch_size = 64, validation_split=0.1, callbacks=[lr_scheduler])

# 지난 3개월 각 카테고리별 소비량을 개월별로 출력
last_data = df_scaled[-seq_length:].values
predicted = []

for _ in range(30):  # Adjust the number of months you want to predict
    input_data = last_data[np.newaxis, ...]
    prediction = model.predict(input_data)
    predicted.append(prediction[0])

    # Include the predicted data for the next month
    last_data = np.concatenate((last_data[1:], prediction), axis=0)

# Inverse transform and aggregate predicted results
predicted = np.array(predicted)
predicted_real = scaler.inverse_transform(predicted)

# Aggregate predicted monthly consumption for each category
categories = df.columns[1:]
monthly_predicted = []

for i, month in enumerate(predicted_real.T):  # Transpose to iterate over categories
    monthly_sum = month.sum()
    monthly_predicted.append(monthly_sum)

print("Total Monthly Predicted Consumption:")
for i, category in enumerate(categories):
    print(f"{category}: {monthly_predicted[i]:.2f}")

In [ ]:
save_model(model, 'lstm_first.h5')

In [ ]:
#pip freeze > requirements2.txt

In [ ]:
pip install fbprophet

In [ ]:
from fbprophet import Prophet

# 결과를 저장할 딕셔너리
predictions = {}

# 각 카테고리별로 모델 학습 및 예측
for category in df.columns[1:]:
    temp_df = df[['거래일시', category]].rename(columns={'거래일시': 'ds', category: 'y'})

    # Prophet 모델 학습
    model = Prophet(yearly_seasonality=True, daily_seasonality=True)
    model.fit(temp_df)

    # 다음 달을 예측하기 위한 미래 날짜 데이터프레임 생성
    future = model.make_future_dataframe(periods=30)

    # 예측 수행
    forecast = model.predict(future)

    # 예측 결과 저장
    predictions[category] = forecast['yhat'][-30:].sum()  # 다음 달의 예상 합계

# 결과 출력
for category, value in predictions.items():
    print(f"{category}: {value:.2f}")
